In [78]:
import pandas as pd
import numpy as np
df = pd.read_csv('/home/hjjung/ailab/anomaly_TFT/templates/GHL/debug_train.csv', index_col=False)

In [79]:
# df.reset_index(drop=True)
df.head()

,Time,RT_level_ini,RT_temperature.T,HT_temperature.T,RT_level,out_valve_act,dT_rand,inv_valve_act,limiter.y,inj_valve_act,Relaxing.active,boundary.m_flow_in,dir_valve_act,dt_rand,C_level,C_temperature.T,heater_act,HT_level,limiter1.y,dL_rand
0,0.0,0.0,283.149994,283.149994,0.5,0.0,0.0,0.0,283.0,0.0,0.0,0.0,0.0,0.0,0.01,273.149994,0.0,0.1,100.0,0.0
1,1.0,0.0,283.149994,283.163330,0.5,0.0,0.0,0.0,283.0,0.0,0.0,0.0,0.0,0.0,0.01,273.149994,0.0,0.1,100.0,0.0
2,2.0,0.0,283.149994,283.176666,0.5,0.0,0.0,0.0,283.0,0.0,0.0,0.0,0.0,0.0,0.01,273.149994,0.0,0.1,100.0,0.0
3,3.0,0.0,283.149994,283.189972,0.5,0.0,0.0,0.0,283.0,0.0,0.0,0.0,0.0,0.0,0.01,273.149994,0.0,0.1,100.0,0.0
4,4.0,0.0,283.149994,283.203247,0.5,0.0,0.0,0.0,283.0,0.0,0.0,0.0,0.0,0.0,0.01,273.149994,0.0,0.1,100.0,0.0


In [34]:
from core.configuration import CONFIGS
import enum

config = CONFIGS['GHL']()

data = df.copy()
print(data.shape)
features = config.features
class DataTypes(enum.IntEnum):
    """Defines numerical types of each culumn."""
    CONTINUOUS = 0
    CATEGORICAL = 1
    DATE = 2
    STR = 3
DTYPE_MAP = {
    DataTypes.CONTINUOUS : np.float64,
    DataTypes.CATEGORICAL : np.int64,
    DataTypes.DATE : 'datetime64[ns]',
    DataTypes.STR : str
}

col_dtypes = {v.name:DTYPE_MAP[v.feature_embed_type] for v in features}
data = data[set(x.name for x in features)]
data = data.astype(col_dtypes)
# print(col_dtypes)
# data = data[['RT_level','RT_temperature.T','HT_level','HT_temperature.T','inj_valve_act','heater_act']]
example_length = 35
stride = 1

(100, 20)


/tmp/ipykernel_2097232/465736950.py:23: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  data = data[set(x.name for x in features)]


In [64]:
print(data.values[0:3])

[[  0.           0.           0.5        283.1499939  283.1499939 ]
 [  0.           0.           0.5        283.1499939  283.16333008]
 [  0.           0.           0.5        283.1499939  283.17666626]]


In [53]:
def standard_scaler(data, mu_sigma_values):
    data_scaled = data.copy()

    if len(mu_sigma_values) == 0:
        #data.shape[1] = num_column
        for i in range(data.shape[1]):
            values = data[:, i]
            mean = np.mean(values)
            std = np.std(values)
            if std == 0:
                std = 1
            mu_sigma_values.append([mean, std])
            print(mean)
            print(std)
            values_scaled = (values - mean) / std
            data_scaled[:, i] = values_scaled
    else:
        for i in range(data.shape[1]):
            values = data[:, i]

            values_scaled = (values - mu_sigma_values[i][0]) / mu_sigma_values[i][1]
            data_scaled[:, i] = values_scaled

    return data_scaled, mu_sigma_values

In [54]:
mu_sigma_values = []
data_scaled, mu_sigma_values = standard_scaler(data.values, mu_sigma_values)

0.0
1
0.0
1
0.5
1
283.14999389600007
5.684341886080802e-14
283.78245971676006
0.3609050732018216


In [59]:
print(data_scaled.shape)

(100, 5)


In [66]:
def stride_series(data, example_length, stride):
    series = []
    for i in range(0,len(data) - example_length, stride):
        series.append(data[i: (i + example_length)])
        # print(len(data[i: (i + example_length)]))
    for i in range(len(series)):
        if len(series[i]) != 35:
            print(len(series[i]))
    series = np.array(series)
    
    return series

In [67]:
series = stride_series(data.values, example_length, stride)

In [69]:
print(series)

[[[  0.           0.           0.5        283.1499939  283.1499939 ]
  [  0.           0.           0.5        283.1499939  283.16333008]
  [  0.           0.           0.5        283.1499939  283.17666626]
  ...
  [  0.           0.           0.5        283.1499939  283.56817627]
  [  0.           0.           0.5        283.1499939  283.58096313]
  [  0.           0.           0.5        283.1499939  283.59371948]]

 [[  0.           0.           0.5        283.1499939  283.16333008]
  [  0.           0.           0.5        283.1499939  283.17666626]
  [  0.           0.           0.5        283.1499939  283.18997192]
  ...
  [  0.           0.           0.5        283.1499939  283.58096313]
  [  0.           0.           0.5        283.1499939  283.59371948]
  [  0.           0.           0.5        283.1499939  283.60647583]]

 [[  0.           0.           0.5        283.1499939  283.17666626]
  [  0.           0.           0.5        283.1499939  283.18997192]
  [  0.           